# keras CNN

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [2]:
from keras.layers import Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
# import tensorflow as tf

# config = tf.ConfigProto(allow_soft_placement=True)
# config.gpu_options.allocator_type = 'BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.40
# config.gpu_options.allow_growth=True

## Data Load

In [4]:
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# normalize 归一化
# 注意：这里如果不归一化，效果特别差
X_train = X_train.reshape(-1, 1, 28, 28)/255.
X_test = X_test.reshape(-1, 1, 28, 28)/255.

# one-hot 编码
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

print(X_train[1].shape)
print(y_train[0])

print()
print(X_train.shape)
print(y_train.shape)

(1, 28, 28)
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]

(60000, 1, 28, 28)
(60000, 10)


## build the CNN

In [5]:
model = Sequential()

# conv layer 1 output shape(32, 28, 28)
model.add(Convolution2D(
        filters=32,  # number of filter, 滤波器的个数（卷积核）
        kernel_size=5,  # number of row， 滤波器的大小
        strides=1,
        padding='same',  # padding method
        batch_input_shape=(None, 1, 28, 28),  # picture height & width
        data_format='channels_first',
))

model.add(Activation('relu'))

# poolint layer 1(max pooling output shape(64, 7, 7))
model.add(
    MaxPooling2D(
    pool_size=2, 
    strides=2,
    padding='same',  # padding method
    data_format='channels_first',
))

In [6]:
# conv layer 2 output shape(64, 14, 14)
model.add(Convolution2D(
        filters=64,  # number of filter, 滤波器的个数（卷积核）
        kernel_size=5,  # number of row， 滤波器的大小
        strides=1,
        padding='same',  # padding method
        data_format='channels_first',
))

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first'))

In [7]:
# fully conneted layer 1 input shape(64*7*&7)=(3136), output shape(1024)
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))

# fully conneted layer 2 to shape(10) for 10 classes
model.add(Dense(10))
model.add(Activation('softmax'))

## Another  to define your optimizer

In [8]:
adam = Adam(lr=1e-4)

model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Train

In [9]:
%%time
print('Training ----------------------')
model.fit(X_train, y_train, epochs=1, batch_size=32)

Training ----------------------
Epoch 1/1
60000/60000 [==============================] - 137s - loss: 0.2058 - acc: 0.9406   
Wall time: 2min 18s


## Test

In [11]:
%%time
print('Testing ----------------------')

# evaluate the model with the metrics we defined earlier[accuracy]
loss, accuracy = model.evaluate(X_test, y_test)

print('\n')
print('test loss:\t', loss)
print('test accuracy:\t', accuracy)

Testing ----------------------
 9920/10000 [============================>.] - ETA: 0s

test loss:	 0.0743820021366
test accuracy:	 0.9753
Wall time: 4.74 s


## 实验结果说明
- 训练时候的batch_size设置为32比64要好

> batch_size=32: 0.9753

> batch_size=64: 0.9686

- 数据预处理时候，一定要归一化。对实验结果影响超级大